In [2]:
import pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *

In [3]:
spark_session = SparkSession.builder \
    .config('spark.sql.extension', 'io.delta.sql.DeltaSparkSessionExtension') \
    .config('spark.jars.packages', 'io.delta:delta-core_2.12:1.0.0,software.amazon.awssdk:bundle:2.15.40,software.amazon.awssdk:url-connection-client:2.15.40,org.apache.spark:spark-avro_2.12:3.2.1,com.amazonaws:aws-java-sdk-bundle:1.12.200,com.amazonaws:aws-java-sdk-pom:1.12.200,org.apache.hadoop:hadoop-aws:3.2.3,com.amazonaws:aws-java-sdk-core:1.12.200,org.apache.spark:spark-sql_2.12:3.2.1') \
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog') \
    .config('hive.metastore.client.factory.class', 'com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory') \
    .config('hive.metastore.schema.verification', 'false') \
    .config('spark.delta.logStore.class', 'org.apache.spark.sql.delta.storage.S3SingleDriverLogStore') \
    .config('spark.sql.hive.convertMetastoreParquet', 'false') \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config("spark.hadoop.fs.s3a.impl", 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.acl.default", 'BucketOwnerFullControl') \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
com.amazonaws#aws-java-sdk-pom added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-core added as a dependency
org.apache.spark#spark-sql_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2f4a49c7-9759-470a-ae82-5e06aac494fa;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in cent

In [4]:
from delta.tables import DeltaTable

In [5]:
# Create a DataFrame
inputDF = spark_session.createDataFrame(
    [
         ("Italy", "11", "1111", "table", 5, 1648126827, "2015/05/02", "2015", "05", "02"),
         ("Spain", "22", "2222", "chair", 5, 1648126827, "2015/05/02", "2015", "05", "02")        
    ],
    ["marketplace", "review_id", "customer_id", "product_title", "star_rating", "timestamp", "review_date", "year", "month", "day" ]
)

# Print Schema of inputDF 
inputDF.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



In [6]:
# inputDF.select(col("review_date"),to_date(col("review_date"),"yyyy/MM/dd").alias("review_date"))

In [7]:
spark_session.sql(
  "create table if not exists default.amazon_customer_review_delta ( \
      marketplace string, \
      review_id string, \
      customer_id string, \
      product_title string, \
      star_rating int, \
      timestamp long , \
      review_date string, \
      year string, \
      month string, \
      day string \
      ) \
      using delta \
      location 's3a://pyspark-format-010953350464-ap-southeast-2/amazon_customer_review_delta/' \
      partitioned by (year,month,day);"
)

Py4JJavaError: An error occurred while calling o50.sql.
: java.lang.NoSuchMethodError: org.apache.spark.sql.execution.datasources.parquet.ParquetSchemaConverter$.checkFieldNames(Lscala/collection/Seq;)V
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.org$apache$spark$sql$delta$catalog$DeltaCatalog$$createDeltaTable(DeltaCatalog.scala:129)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.createTable(DeltaCatalog.scala:196)
	at org.apache.spark.sql.execution.datasources.v2.CreateTableExec.run(CreateTableExec.scala:42)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:49)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution

In [8]:
inputDF \
.write \
.format('delta') \
.mode('overwrite') \
.partitionBy('year,month,day') \
.save('s3a://pyspark-format-010953350464-ap-southeast-2/amazon_customer_review_delta')

22/04/20 04:26:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Py4JJavaError: An error occurred while calling o99.save.
: java.lang.AbstractMethodError: Method org/apache/spark/sql/delta/commands/WriteIntoDelta.org$apache$spark$sql$catalyst$plans$logical$Command$_setter_$nodePatterns_$eq(Lscala/collection/Seq;)V is abstract
	at org.apache.spark.sql.delta.commands.WriteIntoDelta.org$apache$spark$sql$catalyst$plans$logical$Command$_setter_$nodePatterns_$eq(WriteIntoDelta.scala)
	at org.apache.spark.sql.catalyst.plans.logical.Command.$init$(Command.scala:38)
	at org.apache.spark.sql.delta.commands.WriteIntoDelta.<init>(WriteIntoDelta.scala:53)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:154)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala